# Merging all the csv for 2019 and check the title matching with recordlinkage

In [129]:
import pandas as pd
from fuzzywuzzy import fuzz
import recordlinkage

#reading all 4 csv file
imdb = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/IMDB_clean.csv')
omdb = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/Omdb_clean.csv')
mojo = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/mojo_clean.csv')
rotten = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/Rotten_clean.csv')

#renaming the columns name for better match also change the type of the runtime as int for rotten csv
rotten = rotten.rename(columns = {'movie_title':'Title'})
mojo = mojo.rename(columns = {'title':'Title'})
rotten['runtime']=rotten['runtime'].astype(int)

#merge imdb with omdb on Imdb_id - only this two csv have a columns with unique value and can be merge on it
df = imdb.merge(omdb,left_on='imdb_title_id',right_on='imdbID',how='inner')

# drop unwanted columns and rearenge the columns for better visualization
df.drop(columns=['Unnamed: 0_y','imdbID','Released','Unnamed: 0_x','Director','Year'],inplace=True)
col = df.columns.tolist()
new_col=['imdb_title_id','Title','title','director','year','date_published' ,'Runtime','avg_vote','total_votes',
         'us_voters_rating','us_voters_votes','non_us_voters_rating','non_us_voters_votes','females_allages_avg_vote',
         'females_allages_votes','males_allages_avg_vote','males_allages_votes','Metascore']
df = df[new_col]
df = df.rename(columns={'title':'Alternative Title','avg_vote':'imdb_avg_vote','total_votes':'imdb_total_votes',
                       'us_voters_rating':'imdb_us_voters_rating','us_voters_votes':'imdb_us_voters_votes',
                       'non_us_voters_rating':'imdb_non_us_voters_rating','non_us_voters_votes':'imdb_non_us_voters_votes',
                       'females_allages_avg_vote':'imdb_females_avg_vote','females_allages_votes':'imdb_females_votes',
                       'males_allages_avg_vote':'imdb_males_avg_vote','males_allages_votes':'imdb_males_votes'})

#fixing the rantime columns 
df['Runtime'] = df['Runtime'].str.replace(' min','')
df['Runtime'] = df['Runtime'].astype(int)


#merge previous dataframe with rotten dataframe on title columns and print the matches number
df1 = df.merge(rotten,left_on='Title',right_on='Title',how='inner')
print('rows matches before merge to mojo',df1.shape)


#merge with mojo box office data
movie_df_final = df1.merge(mojo,left_on='Title',right_on='Title',how='inner')

#drop and rearrange columns for better visualization
movie_df_final.drop(columns=['Unnamed: 0_x','year_y','original_release_date','runtime','Unnamed: 0_y','year','directors'],inplace=True)
movie_df_final = movie_df_final.rename(columns={'year_x':'Year','audience_rating':'tomato_audience_rating','audience_count':'tomato_audience_vote_count'})
movie_df_final['Domestic_Lifetime_Gross'] = movie_df_final['Domestic_Lifetime_Gross'].str.replace('-','0')
movie_df_final['Domestic_Lifetime_Gross'] = movie_df_final['Domestic_Lifetime_Gross'].astype(int)
movie_df_final[['tomato_audience_vote_count','tomato_audience_rating','tomatometer_count','tomatometer_rating','Metascore','imdb_males_votes','imdb_females_votes','imdb_non_us_voters_votes','imdb_us_voters_votes','imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']]=movie_df_final[['tomato_audience_vote_count','tomato_audience_rating','tomatometer_count','tomatometer_rating','Metascore','imdb_males_votes','imdb_females_votes','imdb_non_us_voters_votes','imdb_us_voters_votes','imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']].astype(float)
movie_df_final[['imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']]= movie_df_final[['imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']]*10
movie_df_final[['tomato_audience_vote_count','tomato_audience_rating','tomatometer_count','tomatometer_rating','Metascore','imdb_males_votes','imdb_females_votes','imdb_non_us_voters_votes','imdb_us_voters_votes','imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']]=movie_df_final[['tomato_audience_vote_count','tomato_audience_rating','tomatometer_count','tomatometer_rating','Metascore','imdb_males_votes','imdb_females_votes','imdb_non_us_voters_votes','imdb_us_voters_votes','imdb_avg_vote','imdb_us_voters_rating','imdb_non_us_voters_rating','imdb_females_avg_vote','imdb_males_avg_vote']].astype(int)


#print information about the final matches ( only for 2019 year)
print(movie_df_final.shape)
print(movie_df_final.columns)
print(movie_df_final.head())
print(movie_df_final.dtypes)
print(movie_df_final.describe())

movie_df_final.to_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/processed/movie_df_final.csv')



rows matches before merge to mojo (843, 27)
(676, 25)
Index(['imdb_title_id', 'Title', 'Alternative Title', 'director', 'Year',
       'date_published', 'Runtime', 'imdb_avg_vote', 'imdb_total_votes',
       'imdb_us_voters_rating', 'imdb_us_voters_votes',
       'imdb_non_us_voters_rating', 'imdb_non_us_voters_votes',
       'imdb_females_avg_vote', 'imdb_females_votes', 'imdb_males_avg_vote',
       'imdb_males_votes', 'Metascore', 'tomatometer_rating',
       'tomatometer_count', 'tomato_audience_rating',
       'tomato_audience_vote_count', 'Worldwide_Lifetime_Gross',
       'Domestic_Lifetime_Gross', 'Foreign_Lifetime_Gross'],
      dtype='object')
  imdb_title_id                                        Title  \
0     tt2226597                      The Mountain Between Us   
1     tt2231461                                      Rampage   
2     tt2239822  Valerian and the City of a Thousand Planets   
3     tt2249081                                 The Forgiven   
4     tt2250912   

# use recordlinkage to see if merging on title is valid merge or not

In [123]:
import recordlinkage

print(rotten.shape)
indexer = recordlinkage.Index()
indexer.block('Title')
pairs = indexer.index(df,rotten)
compare_cl = recordlinkage.Compare()
compare_cl.exact('year','year')
compare_cl.string('Title','Title',threshold=0.85)
compare_cl.string('Alternative Title','Title',threshold=0.70)
compare_cl.string('director','directors',threshold=0.85)
pot_matches = compare_cl.compute(pairs,df,rotten)
matches = pot_matches[pot_matches.sum(axis=1)>=2]
print(matches)

        

(1751, 10)
           0    1    2    3
0    1496  1  1.0  0.0  1.0
1    1085  1  1.0  0.0  1.0
2    1667  1  1.0  0.0  1.0
3    1387  0  1.0  0.0  1.0
4    1216  1  1.0  1.0  1.0
...       ..  ...  ...  ...
1656 103   0  1.0  0.0  1.0
1659 485   1  1.0  0.0  1.0
1668 1554  1  1.0  1.0  1.0
1669 731   1  1.0  1.0  1.0
1671 1356  1  1.0  1.0  1.0

[841 rows x 4 columns]


This test shows that merging on title is valid , I got 843 matches merging just the title columns and 841 by using recordlinkage and use 85% title character match.